In [11]:
import numpy as np
import collections
import time



class MaxPoolingFactory:
    def __init__(self, matrix):

        # if type of matrix is numpy matrix, set handler to NumPyMaxPooling
        if isinstance(matrix, np.ndarray):
            self.handler = NumPyMaxPooling(matrix)

        # else, set handler to ListMaxPooling
        else:
            self.handler = ListMaxPooling(matrix)

    def get_handler(self):
        return self.handler

class MaxPooling:
    def __init__(self, matrix):
        self.matrix = matrix

    def maxSlidingWindow(self, nums, k):
      pass

    def max_pool(self,window_size):
      pass

class NumPyMaxPooling(MaxPooling):
    def __init__(self, matrix):
        self.matrix = matrix

    def maxSlidingWindow(self, nums, k):
      """
      Return the windows of the sliding window
      :type nums: np.ndarray
      :type k: int
      :rtype: np.ndarray
      """

      d = collections.deque()
      nums_len = len(nums)
      out = np.zeros(nums_len, dtype=int)  # Pre-allocate a NumPy array with zeros

      # Variable to track the index for inserting the maximum value
      output_index = 0

      for i, n in enumerate(nums):
          # Remove all elements smaller than the current element
          while d and nums[d[-1]] < n:
              d.pop()

          # Add the current element's index to the deque
          d.append(i)

          # Remove the element that is outside the sliding window
          if d[0] == i - k:
              d.popleft()

          # Append the maximum element of the current window to the output
          if i >= k - 1:
              out[output_index] = nums[d[0]]
              output_index += 1

      return out



    def max_pool(self, window_size):
        # If window_size <= 1, return matrix as it is
        if window_size <= 1:
            return self.matrix

        # Step 1: Apply sliding window horizontally across rows
        if window_size >= len(self.matrix[0]):
            row_max = np.max(self.matrix, axis=1).reshape(-1, 1)
        else:
            # Sliding window over axis 1 (columns), use window_size only
            row_windows = np.lib.stride_tricks.sliding_window_view(self.matrix, window_shape=window_size, axis=1)
            row_max = np.max(row_windows, axis=2)

        # Step 2: Transpose the result of row-wise max pooling
        row_max_transposed = row_max.T

        # Step 3: Apply sliding window over the transposed matrix (which corresponds to columns in the original matrix)
        if window_size >= len(row_max_transposed[0]):
            col_max = np.max(row_max_transposed, axis=1).reshape(-1, 1)
        else:
            # Sliding window over axis 1 (columns of transposed), use window_size only
            col_windows = np.lib.stride_tricks.sliding_window_view(row_max_transposed, window_shape=window_size, axis=1)
            col_max = np.max(col_windows, axis=2)

        # Step 4: Transpose the final result back to get the original dimensions
        final_output = col_max.T

        return final_output

class ListMaxPooling(MaxPooling):
    def __init__(self, matrix):
        self.matrix = matrix

    def maxSlidingWindow(self,nums, k):
      """
      Return the windows of the sliding window
      :type nums: List[int]
      :type k: int
      :rtype: List[int]
      """

      d = collections.deque()
      out = []
      nums_len = len(nums)

      for i, n in enumerate(nums):

          while d and nums[d[-1]] < n:
              d.pop()

          d.append(i)

          if d[0] == i - k:
              d.popleft()

          if i>=k-1:
              out.append(nums[d[0]])

      # Padding the output array to make it len(input matrix)
      for i in range(len(nums) - len(out)):
        out.append(0)
      return out

    def max_pool(self, window_size):

      if(len(self.matrix) == 0):
        return []

      if window_size <= 1:
        return self.matrix

      if window_size >= len(self.matrix):
        # Find max of matrix and return it
        ret = [[0]*len(self.matrix) for i in range(len(self.matrix))]
        ret[0][0] = max(map(max, self.matrix))
        return ret


      output_matrix = []
      for i in range(len(self.matrix)):
        output_matrix.append(self.maxSlidingWindow(self.matrix[i], window_size))

      # Transpose the list of lists
      output_matrix = list(map(list, zip(*output_matrix)))

      # Calculate sliding window of rows of output_matrix
      for i in range(len(output_matrix)):
        output_matrix[i] = self.maxSlidingWindow(output_matrix[i], window_size)

      # Transpose the list of lists
      output_matrix = list(map(list, zip(*output_matrix)))


      return output_matrix

    def max_pool_naive(self, matrix, window_size):
      n = len(matrix)

      output_size = n - window_size + 1

      output = [[0 for _ in range(output_size)] for _ in range(output_size)]

      if(window_size == 1):
        return matrix

      if(window_size >= len(matrix)):
        return matrix

      for i in range(output_size):
          for j in range(output_size):
              window = [matrix[x][j:j+window_size] for x in range(i, i+window_size)]
              max_in_window = max(max(row) for row in window)
              output[i][j] = max_in_window

      # pad output to make it original size
      cols = len(matrix) - output_size
      for i in range(output_size):
        for j in range(cols):
          output[i].append(0)
      for i in range(cols):
        output.append([0]*len(matrix[0]))

      return output



In [12]:

a = np.matrix([[3, 4, 2, 1], [1, 5, 4, 6], [3, 6, 7, 2], [3, 2, 5, 4]])

maxPoolFactory = MaxPoolingFactory(a)
maxPoolFactory.get_handler().max_pool(3)


matrix([[7]])

# **Test Cases**


In [13]:
matrix = [[3, 4, 2, 1], [1, 5, 4, 6], [3, 6, 7, 2], [3, 2, 5, 4]]
window_size = 2

maxPoolFactory = MaxPoolingFactory(matrix)
output = maxPoolFactory.get_handler().max_pool(window_size)

expected_output = [[5, 5, 6, 0], [6, 7, 7, 0], [6, 7, 7, 0], [0, 0, 0, 0]]

assert output == expected_output, "Output does not match expected output"
print("OK")


OK


In [14]:
matrix = [
    [1, 3, 2],
    [4, 6, 5],
    [7, 8, 9]
]
window_size = 2
maxPoolFactory = MaxPoolingFactory(matrix)
output = maxPoolFactory.get_handler().max_pool(window_size)
expected_output = [
    [6, 6, 0],
    [8, 9, 0],
    [0, 0, 0]
]
assert output == expected_output, "Output does not match expected output"
print("OK")


OK


In [15]:
matrix = [
    [1, 3, 2],
    [4, 6, 5],
    [7, 8, 9]
]

window_size = 3
maxPoolFactory = MaxPoolingFactory(matrix)
output = maxPoolFactory.get_handler().max_pool(window_size)
expected_output = [
    [9, 0, 0],
    [0, 0, 0],
    [0, 0, 0]
]
assert output == expected_output, "Output does not match expected output"
print("OK")

OK


In [16]:
matrix = [
    [1, 3, 2, 5],
    [4, 6, 5, 7],
    [7, 8, 9, 10],
    [11, 12, 13, 14]
]

window_size = 1
maxPoolFactory = MaxPoolingFactory(matrix)
output = maxPoolFactory.get_handler().max_pool(window_size)
expected_output = [
    [1, 3, 2, 5],
    [4, 6, 5, 7],
    [7, 8, 9, 10],
    [11, 12, 13, 14]
]
assert output == expected_output, "Output does not match expected output"
print("OK")

OK


In [17]:
matrix = []
window_size = 2
maxPoolFactory = MaxPoolingFactory(matrix)
output = maxPoolFactory.get_handler().max_pool(window_size)
expected_output = []
assert output == expected_output, "Output does not match expected output"
print("OK")

OK


In [18]:
matrix = [
    [42]
]
window_size = 1
maxPoolFactory = MaxPoolingFactory(matrix)
output = maxPoolFactory.get_handler().max_pool(window_size)
expected_output = [[
    42]
]
assert output == expected_output, "Output does not match expected output"
print("OK")

OK


# **Interface for Max-Pooling**

In [19]:
n = int(input("Enter the size of matrix: "))

matrix = []
for i in range(n):
    row = list(map(int, input().split()))
    matrix.append(row)

window_size = int(input("Enter the window size: "))
maxPoolFactory = MaxPoolingFactory(matrix)

start_time = time.time()
output = maxPoolFactory.get_handler().max_pool(window_size)

print(output)
print("--- %s seconds ---" % (time.time() - start_time))

print("Naive implementation")

start_time = time.time()
print(maxPoolFactory.get_handler().max_pool_naive(matrix, window_size))
print("--- %s seconds ---" % (time.time() - start_time))


Enter the size of matrix: 4
1 2 3 4
5 6 7 8 
9 10 11 12
13 14 15 16
Enter the window size: 3
[[11, 12, 0, 0], [15, 16, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]
--- 0.0003490447998046875 seconds ---
Naive implementation
[[11, 12, 0, 0], [15, 16, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]
--- 0.0001709461212158203 seconds ---
